# Setup : Enable necessary APIs
*   Dataform API
*   BigQuery API
*   BigQuery Connection API
*   Vertex AI API
*   Compute Engine API
*   Cloud Build API
*   Cloud Functions API
*   Cloud Logging API
*   Cloud Pub/Sub API
*   Cloud Run Admin API
*   Cloud Speech-to-Text API
*   Secret Manager API

*API List: https://developers.google.com/apis-explorer*

*Validate API Activation: https://console.cloud.google.com/apis/dashboard*

In [ ]:
#APIs
!gcloud services enable dataform.googleapis.com bigquery.googleapis.com bigqueryconnection.googleapis.com aiplatform.googleapis.com compute.googleapis.com cloudbuild.googleapis.com cloudfunctions.googleapis.com logging.googleapis.com pubsub.googleapis.com run.googleapis.com speech.googleapis.com secretmanager.googleapis.com

# Setup : Create a Connection to an External Data Source in BigQuery and Grant the Permissions to use Vertex AI and Invoke Cloud Functions to the LLM Service Account.

*Roles List: https://cloud.google.com/iam/docs/understanding-roles*

*Validate Permitted Roles: https://console.cloud.google.com/iam-admin/iam*

In [ ]:
#connection
!bq mk --connection --location=eu --connection_type=CLOUD_RESOURCE llm_conn

In [ ]:
import os
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

SERVICE_ACCT = !bq show --format=prettyjson --connection eu.llm_conn | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
SERVICE_ACCT_MEMBER = 'serviceAccount:' + SERVICE_ACCT_EMAIL

###################################
#AI platform
gcloud_aiplatform = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{SERVICE_ACCT_MEMBER}" --role="roles/aiplatform.user"
"""
print(gcloud_aiplatform)
!$gcloud_aiplatform

###################################
# 1st gen cloud functions
gcloud_cloudfunctionsinvoker = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{SERVICE_ACCT_MEMBER}" --role="roles/cloudfunctions.invoker"
"""
print(gcloud_cloudfunctionsinvoker)
!$gcloud_cloudfunctionsinvoker

###################################
#2nd gen cloud functions
gcloud_runinvoker = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{SERVICE_ACCT_MEMBER}" --role="roles/run.invoker"
"""
print(gcloud_runinvoker)
!$gcloud_runinvoker

###################################
#IAM roles validation
!gcloud projects get-iam-policy $PROJECT_ID --flatten="bindings[].members" --format="table(bindings.role)" --filter="bindings.members:$SERVICE_ACCT_MEMBER"

# Setup : Deploy the Cloud Function and Grant the Corresponding Permissions to the LLM Service Account

*More Info:*
- *https://cloud.google.com/functions/docs/create-deploy-gcloud*
- *https://cloud.google.com/sdk/gcloud/reference/functions/deploy*

*Validate Cloud Functions Installed: https://console.cloud.google.com/functions/list*

In [65]:
CLOUD_FUNCTION_NAME = "bigquery_external_function"
#the cloud function zip must be in the following directory to be deployed (it must include both main.py and requirements.txt files)
FUNCTION_ZIP_GCS_URI = f"gs://{PROJECT_ID}/{CLOUD_FUNCTION_NAME}.zip"
REGION = "europe-west4"

In [ ]:
#deploy 2nd gen cloud functions
gcloud_deploycloudfunction = f"""
gcloud functions deploy {CLOUD_FUNCTION_NAME} \
--gen2 \
--runtime=python312 \
--region={REGION} \
--source={FUNCTION_ZIP_GCS_URI} \
--entry-point={CLOUD_FUNCTION_NAME} \
--set-env-vars PROJECT_ID={PROJECT_ID},ENV_CLOUD_FUNCTION_REGION={REGION} \
--no-allow-unauthenticated \
--trigger-http
"""
print(gcloud_deploycloudfunction)
!$gcloud_deploycloudfunction

In [ ]:
#grant invoke permissions to 2nd gen cloud functions
gcloud_runinvoker_external_function = f"""
gcloud functions add-invoker-policy-binding {CLOUD_FUNCTION_NAME} --region="{REGION}" --no-user-output-enabled --member="{SERVICE_ACCT_MEMBER}"
"""
print(gcloud_runinvoker_external_function)
!$gcloud_runinvoker_external_function

# Setup : Grant the necessary permissions to the Dataform Service Account after repository creation


In [ ]:
import os
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

DATAFORM_SERVICE_ACCT = !gcloud projects get-iam-policy $PROJECT_ID --flatten="bindings[].members" --format="table(bindings.members)" --filter="bindings.members:gcp-sa-dataform" | grep "serviceAccount" | cut -d ':' -f 2
DATAFORM_SERVICE_ACCT_EMAIL = DATAFORM_SERVICE_ACCT[-1]
DATAFORM_SERVICE_ACCT_MEMBER = 'serviceAccount:' + DATAFORM_SERVICE_ACCT_EMAIL

###################################
#Secret access
gcloud_secretmanager = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{DATAFORM_SERVICE_ACCT_MEMBER}" --role="roles/secretmanager.secretAccessor"
"""
print(gcloud_secretmanager)
!$gcloud_secretmanager

###################################
#BigQuery User
gcloud_bigqueryuser = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{DATAFORM_SERVICE_ACCT_MEMBER}" --role="roles/bigquery.user"
"""
print(gcloud_bigqueryuser)
!$gcloud_bigqueryuser

###################################
#Connection Admin
gcloud_connectionadmin = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{DATAFORM_SERVICE_ACCT_MEMBER}" --role="roles/bigquery.connectionAdmin"
"""
print(gcloud_connectionadmin)
!$gcloud_connectionadmin

###################################
#IAM roles validation
!gcloud projects get-iam-policy $PROJECT_ID --flatten="bindings[].members" --format="table(bindings.role)" --filter="bindings.members:$DATAFORM_SERVICE_ACCT_MEMBER"

---
---
---
---
# List/Download/Play sample audios

In [ ]:
%%bigquery
--source table = source.reviews_audios
SELECT REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') as trip_id,*
  FROM EXTERNAL_OBJECT_TRANSFORM(TABLE `source.reviews_audios`, ['SIGNED_URL'])
 LIMIT 5;

In [89]:
trip_id = '30618713' #@param{type:"string"}
params = { "trip_id": trip_id}

In [ ]:
%%bigquery signed_url --params $params
--source table = source.reviews_audios
WITH signed_urls AS (
  SELECT signed_url,
         REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') as trip_id
  FROM EXTERNAL_OBJECT_TRANSFORM(TABLE `source.reviews_audios`, ['SIGNED_URL'])
)
SELECT signed_url
FROM signed_urls
WHERE trip_id = @trip_id;

In [ ]:
import requests

response = requests.get(signed_url['signed_url'][0],stream=True)
with open(f'text_synth_{trip_id}.mp3', 'wb') as f:
  f.write(response.content)

import IPython.display
IPython.display.Audio(f'text_synth_{trip_id}.mp3',rate=16000)